# DataModelDict Class Demonstration

The DataModelDict class is used for handling data models that have equivalent representations in XML, JSON, and Python.  Constructing data models in this way is convenient as it supports compatibility across different software tools, such as different types of databases.

The DataModelDict class:

- is a child of OrderedDict,
- has methods for converting to/from XML and JSON, 
- has methods for searching through elements, and
- has methods that help with constructing and interacting with compliant data models.

## Designing Compatible Data Models

Some considerations need to be taken into account for designing data models that allow for exact reversible transformations between the three formats:

- Valid, full XML requires that there is exactly one root element.  In other words, the top-level DataModelDict of a data model   can have only one key.
- Do not use lists of lists for representing data.  The XML conversions are only reversible for lists of values or lists of    
  dictionaries.  Future updates may allow this.
- Avoid using XML attributes if possible.  While the XML conversions do reversibly handle attributes, it complicates the Python
  and JSON representations.
- Embedded XML content, i.e. "text with &lt;embed&gt;embedded&lt;/embed&gt; content", might not be reversible:

    - If this is in a Python/JSON value, converting to XML gives "text with &amp;lt;embed&amp;gt;embedded&amp;lt;/embed&amp;gt; content". This is reversible.
    - If this is an XML text field, parsing to Python pulls the embedded elements out of the text, which is not reversible!

- XML subelements of the same name within an element should be given consecutively.  When parsed, all values of subelements of the same name are collected together in a list.  This will alter the original order of subelements if matching names were not originally consecutive. 

#### Library imports

In [1]:
# Standard Python libraries
from __future__ import print_function, division
import os
import random

# DataModelDict class
# https://github.com/usnistgov/DataModelDict
from DataModelDict import DataModelDict as DM

# Print DataModelDict version
import DataModelDict
print('DataModelDict version =', DataModelDict.__version__)

DataModelDict version = 0.9.2


## 1. Class Basics 

The DataModelDict is a child class of OrderedDict.  As such, is has all the functionality of OrderedDict and more.

Here, we construct a multi-level demonstration data model using lists and DataModelDicts.

In [2]:
# Create an empty DataModel
model = DM()

# Build model element by element
model['my-data-model'] = DM()

model['my-data-model']['name'] = 'Demo'
model['my-data-model']['author'] = 'Me'

model['my-data-model']['process'] = DM()
model['my-data-model']['process']['Instrument'] = DM()
model['my-data-model']['process']['Instrument']['Name'] = 'Shiny Thing'
model['my-data-model']['process']['Instrument']['Model'] = 'Newest Most\nExpensive'
model['my-data-model']['process']['method'] = 'By the book'

# Assign multiple elements at once
model['my-data-model']['measurement'] = []
for temperature in range(0, 2000, 200):
    measurement = DM([('temperature', DM([('value', temperature),                    
                                          ('unit', 'K')])),
                      ('length',      DM([('value', temperature*random.random()/50), 
                                          ('unit', 'm')]))])
    model['my-data-model']['measurement'].append(measurement) 

## 2. Output Conversion

DataModelDict has methods json() and xml() that return the data model as either of these formats. 

### Conversion from Python to JSON

The Python-JSON conversions use the standard Python JSON library.  In converting from Python to JSON, all elements of the DataModelDict must be an instance of a supported data type (with unicode and long being specific to Python 2).


| Python           | JSON      |
| ---------------- | --------- |
| dict             | object    |
| list, tuple      | array     |
| str, unicode     | string    |
| int, long, float | number    |
| True             | true      |
| False            | false     |
| None             | null      |
| np.nan           | NaN       |
| np.inf           | Infinity  |
| -np.inf          | -Infinity |


As DataModelDict is a child of OrderedDict, it registers as being an instance of dict. Any other objects would first need to be converted to one of these types, e.g. a numpy array would need to be converted to a list. 

In [3]:
# Save DataModelDict as json file by setting fp = file-like object
with open('model.json', 'w') as f:
    model.json(fp=f)

# Print the DataModelDict as a json string. 
# Setting an indent value adds newlines and line indentations
print(model.json(indent=2))

{
  "my-data-model": {
    "name": "Demo", 
    "author": "Me", 
    "process": {
      "Instrument": {
        "Name": "Shiny Thing", 
        "Model": "Newest Most\nExpensive"
      }, 
      "method": "By the book"
    }, 
    "measurement": [
      {
        "temperature": {
          "value": 0, 
          "unit": "K"
        }, 
        "length": {
          "value": 0.0, 
          "unit": "m"
        }
      }, 
      {
        "temperature": {
          "value": 200, 
          "unit": "K"
        }, 
        "length": {
          "value": 3.503408529019528, 
          "unit": "m"
        }
      }, 
      {
        "temperature": {
          "value": 400, 
          "unit": "K"
        }, 
        "length": {
          "value": 2.0722318138461935, 
          "unit": "m"
        }
      }, 
      {
        "temperature": {
          "value": 600, 
          "unit": "K"
        }, 
        "length": {
          "value": 4.773601968546896, 
          "unit": "m"
        }
      

### Conversion from Python to XML

The Python-XML conversions use the xmltodict Python package. The XML content is constructed based on the Python data types 

| Python           | XML              |
| ---------------- | ---------------- |
| dict             | subelement       |
| list, tuple      | repeated element |
| str, unicode     | text             |
| int, long, float | text (from repr) |
| True             | text = True      |
| False            | text = False     |
| None             | empty text field |
| np.nan           | text = NaN       |
| np.inf           | text = Infinity  |
| -np.inf          | text = -Infinity |

Some characters in the XML text fields will also be converted to avoid conflicts.

- XML limited characters such as &lt;, &gt; and &amp; are converted to their HTML entities.
- \n, \t, \r are converted to \\\n, \\\t, and \\\r 

Any dictionary keys starting with '@' will be converted into XML attributes, and the dictionary key '#text' is interpreted as the text value of the element. 

In [4]:
# Save DataModelDict as xml file by setting fp = file-like object
with open('model.xml', 'w') as f:
    model.xml(fp=f)

# Print the DataModelDict as an xml string. 
# Setting an indent value adds newlines and line indentations
print(model.xml(indent=4))

<?xml version="1.0" encoding="utf-8"?>
<my-data-model>
    <name>Demo</name>
    <author>Me</author>
    <process>
        <Instrument>
            <Name>Shiny Thing</Name>
            <Model>Newest Most\nExpensive</Model>
        </Instrument>
        <method>By the book</method>
    </process>
    <measurement>
        <temperature>
            <value>0</value>
            <unit>K</unit>
        </temperature>
        <length>
            <value>0.0</value>
            <unit>m</unit>
        </length>
    </measurement>
    <measurement>
        <temperature>
            <value>200</value>
            <unit>K</unit>
        </temperature>
        <length>
            <value>3.503408529019528</value>
            <unit>m</unit>
        </length>
    </measurement>
    <measurement>
        <temperature>
            <value>400</value>
            <unit>K</unit>
        </temperature>
        <length>
            <value>2.0722318138461935</value>
            <unit>m</unit>
        </leng

## 3. Loading Data Models

DataModelDict has a load() method that reads in xml or json content. The class initilizer also calls load() if the argument is a string or file-like object. Both work if the supplied argument is:

- a string directory path to an xml or json file.

- a string containing xml or json content.

- an open file-like object containing xml or json content.


### Conversion from JSON to Python

The Python-JSON conversions use the standard Python JSON library.  In converting from JSON to Python, the conversions of types is straight-forward.

| JSON          | Python 2      | Python 3      |
| ------------- | ------------- | ------------- |
| object        | DataModelDict | DataModelDict |
| array         | list          | list          |
| string        | unicode       | str           |
| number (int)  | long          | int           |
| number (real) | float         | float         |
| true          | True          | True          |
| false         | False         | False         |
| null          | None          | None          |
| NaN           | np.nan        | np.nan        |
| Infinity      | np.inf        | np.inf        |
| -Infinity     | -np.inf       | -np.inf       |

### Conversion from XML to Python

The Python-XML conversions use the xmltodict Python package.  The text fields will be interpreted based on the following sequential tests:

| XML text         | Python 2 | Python 3 |
| ---------------- | -------- | -------- |
| == 'True'        | True     | True     |
| == 'False'       | False    | False    |
| == ''            | None     | None     |
| == 'NaN'         | np.nan   | np.nan   |
| == 'Infinity'    | np.inf   | np.inf   |
| == '-Infinity'   | -np.inf  | -np.inf  |
| try: int(text)   | long     | int      |
| try: float(text) | float    | float    |
| otherwise        | unicode  | str      |

The reverse conversions are done for the special characters mentioned in the Conversion from Python to XML section above.

Any 'attr' attribute fields are converted to elements named '@attr' and corresponding '#text' elements are created if needed.

In [5]:
# Load from xml file during initilization
model2 = DM('model.xml')
    
# Test that models are equivalent# Load from json file using load()
model2 = DM()
model2.load('model.json')
    
# Test that models are equivalent
print(model.json() == model2.json() and model.xml()  == model2.xml())

True


In [6]:
# Load from json file using load()
model2 = DM()
model2.load('model.json')
    
# Test that models are equivalent
print(model.json() == model2.json() and model.xml()  == model2.xml())

True


In [7]:
# Load from json string during initilization
json_string = model.json()
model2 = DM(json_string)
    
# Test that models are equivalent
print(model.json() == model2.json() and model.xml()  == model2.xml())

True


In [8]:
# Load from xml string using load()
xml_string = model.xml()
model2 = DM()       
model2.load(xml_string)

#test that models are equivalent
print(model.json() == model2.json() and model.xml()  == model2.xml())

True


## 4. Finding and Accessing Elements

A number of methods have been added to DataModelDict to assist in finding, accessing, and modifying the various elements and subelements of a data model.

### 4.1 Index with path lists

Normally, accessing or setting the values contained in a data model consisting of tiered dictionaries and lists requires knowing the full path list beforehand.  This can be tedious and requires that the programmer hard-code the absolute path of any elements of interest.  To improve upon this, values contained in a DataModelDict can be accessed using a _path list_ consisting of a list of indicies.  The terms in the list can be either dictionary keys or list indicies.

In [9]:
# Use indexing to retrieve the instrument name in the standard way
print(model['my-data-model']['process']['Instrument']['Name'])

# Use path list indexing to retrieve the instrument name
path = ['my-data-model', 'process', 'Instrument', 'Name']
print(model[path])

Shiny Thing
Shiny Thing


### 4.2 Find value(s) with key

If you know the key for an element you are interested in but don't know where it is located in the data model, you can access the element's value using the find() and finds() methods.  find() will return a value if the search produces a unique result, and issue an error if no match or multiple matches are found.  finds() returns a list of all values obtained by the search conditions.

In [10]:
# I know the instrument name is the only element with the key Name
print(model.find('Name'))

Shiny Thing


In [11]:
# I want a list of all the measurement elements
measurements = model.finds('measurement')

print(len(measurements), "measurements found, with first measurement being:")
print(measurements[0].json(indent=2))

10 measurements found, with first measurement being:
{
  "temperature": {
    "value": 0, 
    "unit": "K"
  }, 
  "length": {
    "value": 0.0, 
    "unit": "m"
  }
}


Both find() and finds() allow for additional search conditions using dictionary arguments _yes_ and _no_.  Any key-value pairs listed in _yes_ must be found in the element in order for it to be considered a match.  If any key-value pairs listed in _no_ are found in the element, then it is rejected.  

In [12]:
#I want only the length from the measurement with temperature equal to 1600  
temp = DM([('value',1600), ('unit', 'K')])

print(model.find('measurement', yes={'temperature':temp})['length']['value'])

#I want all measurements except for when temperature equals 800
temp = DM([('value',800), ('unit', 'K')])

measurements = model.finds('measurement', no={'temperature':temp})
print(len(measurements), "measurements found that don't have temperature = 800")

7.264976308092109
9 measurements found that don't have temperature = 800


### 4.3 Find path(s) with key

Alternatively, if you want to learn the full path to any elements in unknown locations, you can use the path() and paths() methods.  These behave similarly to find() and finds(), but return path lists instead of values.

In [13]:
# I know the instrument name is the only element with the key Name
path = model.path('Name')
print(path)
print(model[path])

['my-data-model', 'process', 'Instrument', 'Name']
Shiny Thing


The path() and paths() methods also allow for yes and no dictionaries to be used as arguments.

In [14]:
measurement_paths = model.paths('measurement', no={'temperature':temp})
for path in measurement_paths:
    print(path)    

['my-data-model', 'measurement', 0]
['my-data-model', 'measurement', 1]
['my-data-model', 'measurement', 2]
['my-data-model', 'measurement', 3]
['my-data-model', 'measurement', 5]
['my-data-model', 'measurement', 6]
['my-data-model', 'measurement', 7]
['my-data-model', 'measurement', 8]
['my-data-model', 'measurement', 9]


## 5. Treatment of Unbounded Sequences

When converting from XML there is some ambiguity associated with sequences.  The normal parsing method will convert sequences with one element to single values, and sequences with multiple elements to lists.  To help with this, DataModelDict has a couple methods that allow for the handling of elements that may or may not be lists.

The append() method allows for a key-value pair to be added to the DataModelDict.  If the key doesn't already exist, then it is assigned like a regular dictionary.  If the key does exist, the current value is converted into a list if it isn't one and the new value is appended.

The aslist() method returns the value(s) associated with a dictionary key as a list, even if it isn't one.

In [15]:
# Check element value and aslist before key is assigned
print("model['my-data-model'].get('ordinal', None) ->", end='') 
print(model['my-data-model'].get('ordinal', None)) 
print("model['my-data-model'].aslist('ordinal') ->   ", end='') 
print(model['my-data-model'].aslist('ordinal')) 
print() 

# Append a value and check again
print("model['my-data-model'].append('ordinal', 'first')")
model['my-data-model'].append('ordinal', 'first')

print("model['my-data-model'].get('ordinal', None) ->", end='') 
print(model['my-data-model'].get('ordinal', None))
print("model['my-data-model'].aslist('ordinal') ->   ", end='') 
print(model['my-data-model'].aslist('ordinal'))
print() 

# Append a value and check again
print("model['my-data-model'].append('ordinal', 'second')")
model['my-data-model'].append('ordinal', 'second')

print("model['my-data-model'].get('ordinal', None) ->", end='') 
print(model['my-data-model'].get('ordinal', None)) 
print("model['my-data-model'].aslist('ordinal') ->   ", end='') 
print(model['my-data-model'].aslist('ordinal'))
print() 

# Append a value and check again
print("model['my-data-model'].append('ordinal', 'third')")
model['my-data-model'].append('ordinal', 'third')

print("model['my-data-model'].get('ordinal', None) ->", end='') 
print(model['my-data-model'].get('ordinal', None)) 
print("model['my-data-model'].aslist('ordinal') ->   ", end='') 
print(model['my-data-model'].aslist('ordinal')) 

model['my-data-model'].get('ordinal', None) ->None
model['my-data-model'].aslist('ordinal') ->   []

model['my-data-model'].append('ordinal', 'first')
model['my-data-model'].get('ordinal', None) ->first
model['my-data-model'].aslist('ordinal') ->   ['first']

model['my-data-model'].append('ordinal', 'second')
model['my-data-model'].get('ordinal', None) ->['first', 'second']
model['my-data-model'].aslist('ordinal') ->   ['first', 'second']

model['my-data-model'].append('ordinal', 'third')
model['my-data-model'].get('ordinal', None) ->['first', 'second', 'third']
model['my-data-model'].aslist('ordinal') ->   ['first', 'second', 'third']


#### File removal to keep Notebook directory clean.

In [16]:
os.remove('model.json')
os.remove('model.xml')